In [1]:
import numpy as np
import pandas as pd

In [2]:
X=[]
Y=[]
lines=[]

# path='/content/drive/My Drive/Self-driving-car-dataset/'
path='C://Users//HP//Desktop//ML//Self-Driving-Car//driving_dataset//'

with open(path+'data.txt') as file:
    lines=file.readlines()
    
i=0
for line in lines:
    if i==5000:
        break
    i+=1
    w=line.split()
    X.append(w[0])
    Y.append(float(w[1]))

In [3]:
for i,x in enumerate(X):
    X[i]=path+X[i]
    
X[4]

'C://Users//HP//Desktop//ML//Self-Driving-Car//driving_dataset//4.jpg'

In [4]:
import math

for i,y in enumerate(Y):
    Y[i]=(y*math.pi)/180
    
print(Y[89])

0.024609142453120045


In [5]:
X_train=X[0:3500]
X_val=X[3500:]
Y_train=Y[0:3500]
Y_val=Y[3500:]

print("train:",len(X_train),len(Y_train))
print("val:",len(X_val),len(Y_val))

train: 3500 3500
val: 1500 1500


In [6]:
from sklearn.metrics import mean_squared_error

z=[0]*len(Y_val)
mean_squared_error(Y_val, z)

0.6890314625042

In [7]:
import numpy as np
import cv2

def rotate_image(img, smoothed_angle, degrees):
    rows,cols,_ = img.shape
    if smoothed_angle!=degrees:
        smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    
    return dst, smoothed_angle

In [8]:
i=0
import os
wheel=cv2.imread('C://Users//HP//Desktop//ML//Self-Driving-Car//wheel-4.jpg')
smoothed_angle=0
smoothed_angle_actual=0

wheel.shape

(240, 240, 3)

In [9]:
#model.save('m1.h5')
from keras.models import load_model 
model = load_model("m1.h5")

In [11]:

for x in X_train:
    img=cv2.imread(x)/255
    cv2.imshow('frame',img)
    
    rad=model.predict(np.array([img]))[0][0]
    deg=(rad*180)/(math.pi)
    
    w,smoothed_angle=rotate_image(wheel,smoothed_angle,deg)
    cv2.imshow('Prediction',w)
    
    deg2=(Y_train[i]*180)/(math.pi)
    
#     s="Prediction: "+str(deg)+" Actual: "+str(deg2)+"\n"
#     s=s.encode()
#     os.write(1,s)
    w2,smoothed_angle_actual=rotate_image(wheel,smoothed_angle_actual,deg2)
    cv2.imshow('Actual',w2)
    
    i+=1;
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

IndexError: list index out of range